In [1]:
#%reset
# import libraries
import sympy as sym
import numpy as np
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

#sym.init_printing()
from IPython.display import display

In [2]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
m = ConcreteModel() # create model using pyomo
N = 25 # number of nodes (much like samples used between a certain time period)
hm=0.5/N # Scaling factor for time-step
m.N = RangeSet(N) 
m.damping=20

mass_b=2 # Mass of the base
mass_l=1 # Mass of the links
len_b=0.3 # Length of the base
m.l_min=0.2 # Links min length
m.l_max=len_b # Links max length
m.F_max=160 # Maximum force that can be applied
m.t_max=12 # Maximum torque
m.w_max=44 # Max angular velocity (omega)
m.dl_max=3
m.distance=5

print('Yeet')

Yeet


In [3]:
# create symbolic variables
g = sym.symbols('g')
mb,ml = sym.symbols(['m_{body}','m_{leg}']) # mass
lb = sym.symbols(['l_{body}']) # length
ll = sym.symbols(['l_{leg}'])
Inb,Inl = sym.symbols(['I_{body}','I_{leg}']) # moment of intertia

x,z,thLl,thRl,r = sym.symbols(['x','z','\\theta_{Lleg}','\\theta_{Rleg}','r'])
dx,dz,dthLl,dthRl,dr = sym.symbols(['\dot{x}','\dot{z}','\dot{\\theta}_{Lleg}','\dot{\\theta}_{Rleg}','\dot{r}']) 
ddx,ddz,ddthLl,ddthRl,ddr = sym.symbols(['\ddot{x}','\ddot{z}','\ddot{\\theta}_{Lleg}','\ddot{\\theta}_{Rleg}','\ddot{r}'])

q = sym.Matrix([[x],[z],[thLl],[thRl],[r]])
dq = sym.Matrix([[dx],[dz],[dthLl],[dthRl],[dr]])
ddq = sym.Matrix([[ddx],[ddz],[ddthLl],[ddthRl],[ddr]])

# forces
F,tau,GRFx,GRFz = sym.symbols(['F','\\tau','G_x','G_z']) 
print('Yeet')

Yeet


In [4]:
# STEP 1: position vector (COM of each link), as well as other required positions. 
rb = sym.Matrix([[x],
                [z]])

rLl = sym.Matrix([[x + (0.5*r)*sym.cos(thLl)],
                 [z - (0.5*r)*sym.sin(thLl)]])
rRl = sym.Matrix([[x + (0.5*r)*sym.cos(thRl)],
                 [z - (0.5*r)*sym.sin(thRl)]])


footLx = sym.Matrix([x + r*sym.cos(thLl)])
footLz = sym.Matrix([z - r*sym.sin(thLl)]) 
footLz = footLz[0].simplify()
footL_pos= sym.Matrix([[footLx],[0],[footLz]])

footRx = sym.Matrix([x + r*sym.cos(thRl)])
footRz = sym.Matrix([z - r*sym.sin(thRl)]) 
footRz = footRz[0].simplify()
footR_pos= sym.Matrix([[footRx],[0],[footRz]])
print('Done')

Done


In [5]:
# STEP 2: generate expressions for the system space velocities from the jacobians
# the Jacobians
Jb = rb.jacobian(q)
JLl = rLl.jacobian(q)
JRl = rRl.jacobian(q)

vb = Jb*dq
vLl = JLl*dq
vRl = JRl*dq

footLdx = footLx.jacobian(q)*dq #This is symbolic again, so it doesnt change with the collocation
footLdx = footLdx[0].simplify()
footRdx = footRx.jacobian(q)*dq #This is symbolic again, so it doesnt change with the collocation
footRdx = footRdx[0].simplify()
print('Done')

Done


In [10]:
# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb]]) # Mass matrix of body
Ml = sym.Matrix([[ml,ml]]) # Mass matrix of leg(s)

Ib=sym.Matrix([[Inb]]) # Inertia matrix of body
Il=sym.Matrix([[Inl]]) # Inertia matrix of leg(s)

WLl=sym.Matrix([[dthLl]]) # Angular volocity of left-leg
WRl=sym.Matrix([[dthRl]]) # Angular volocity of right-leg

# linear kinetic energy:
T1 = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml*sym.matrix_multiply_elementwise(vLl,vLl) + 0.5*Ml*sym.matrix_multiply_elementwise(vRl,vRl)
# angular kinetic energy:
# this should be a 3*3 matrix of MOI and a vector of angular vel... w'*MOI*w
T2 =  0.5*Il*sym.matrix_multiply_elementwise(WLl,WLl) + 0.5*Il*sym.matrix_multiply_elementwise(WRl,WRl) 

T = sym.Matrix([T1[0]])+sym.Matrix([T2[0]])
# potential energy
V = mb*g*rb[1] + ml*g*rLl[1] + ml*g*rRl[1]
print('Done')

Done
